# 数据处理

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('US-pumpkins.csv')
df.info()
df.head()

In [ ]:
df.shape

In [ ]:
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

In [ ]:
# 定义缺失值比例阈值
missing_threshold = 0.75

# 删除缺失值比例超过阈值的列
drop = df.columns[df.isnull().mean() > missing_threshold]
df = df.drop(drop, axis=1)

missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

In [ ]:
# 填充缺失值
df['Variety'] = df['Variety'].fillna('unknown')
df['Origin'] = df['Origin'].fillna('unknown')
df['Item Size'] = df['Item Size'].fillna('unknown')
df['Color'] = df['Color'].fillna('unknown')
df['Mostly High'] = df['Mostly High'].fillna(df['Mostly High'].mean())
df['Mostly Low'] = df['Mostly Low'].fillna(df['Mostly Low'].mean())

In [ ]:
df.isnull().sum()

In [ ]:
# 将价格列转换为数值类型（去除美元符号并转换为浮点数）
price_columns = ['Low Price', 'High Price', 'Mostly Low', 'Mostly High']
for col in price_columns:
    df[col] = df[col].replace({'\$': '', ',': ''}, regex=True).astype(float)

# 将日期列转换为日期类型
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df.info()
df.head()

In [ ]:
# 识别分类变量列
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print("分类变量列：", categorical_cols)

# 对分类变量进行独热编码
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
print("\n独热编码后数据集形状：", df_encoded.shape)

# 查看编码后数据示例
print("\n独热编码后数据前5行：")
pd.DataFrame(df_encoded.iloc[:5, :]).head()

# 特征选择

In [ ]:
# 计算特征相关性矩阵
correlation = df_encoded.corr()
pd.DataFrame(correlation)

In [ ]:
# 提取与价格相关的特征相关性
price_correlation = correlation['High Price'].sort_values(ascending=False)
print("与High Price相关性强的特征：")
print(price_correlation[abs(price_correlation) > 0.3])

In [ ]:
# 选择与价格相关性强的特征
# selected_features = ['Package_36 inch bins', 'Package_24 inch bins', 'City Name_BOSTON', 'Package_1/2 bushel cartons', 'Variety_MINIATURE']
selected_features = price_correlation[abs(price_correlation) > 0.3].index
print(selected_features)

# 准备特征和目标变量
y = df_encoded['High Price']
X = df_encoded[selected_features].drop(columns = 'High Price')

print(X.shape, y.shape)

In [ ]:
'''# 选择与价格相关性强的特征
selected_features = ['Package_36 inch bins', 'Package_24 inch bins', 'City Name_BOSTON', 'Package_1/2 bushel cartons', 'Variety_MINIATURE']

# 准备特征和目标变量
y = df_encoded['High Price']
X = df_encoded[selected_features]'''

# 模型构建

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 构建GBDT模型
gbdt = GradientBoostingRegressor(n_estimators=100, max_depth=4, random_state=42)

gbdt.fit(X_train, y_train)

y_pred = gbdt.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))
print("R2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('US-pumpkins.csv')
df.info()
df.head()

missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

# 定义缺失值比例阈值
missing_threshold = 0.75

# 删除缺失值比例超过阈值的列
drop = df.columns[df.isnull().mean() > missing_threshold]
df = df.drop(drop, axis=1)

missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

# 填充缺失值
df['Variety'] = df['Variety'].fillna('unknown')
df['Origin'] = df['Origin'].fillna('unknown')
df['Item Size'] = df['Item Size'].fillna('unknown')
df['Color'] = df['Color'].fillna('unknown')
df['Mostly High'] = df['Mostly High'].fillna(df['Mostly High'].mean())
df['Mostly Low'] = df['Mostly Low'].fillna(df['Mostly Low'].mean())

# 将价格列转换为数值类型（去除美元符号并转换为浮点数）
price_columns = ['Low Price', 'High Price', 'Mostly Low', 'Mostly High']
for col in price_columns:
    df[col] = df[col].replace({'\$': '', ',': ''}, regex=True).astype(float)

# 将日期列转换为日期类型
df['Date'] = pd.to_datetime(df['Date'])

# 识别分类变量列
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print("分类变量列：", categorical_cols)

# 对分类变量进行独热编码
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
print("\n独热编码后数据集形状：", df_encoded.shape)

# 查看编码后数据示例
print("\n独热编码后数据前5行：")
pd.DataFrame(df_encoded.iloc[:5, :]).head()

# 计算特征相关性矩阵
correlation = df_encoded.corr()
pd.DataFrame(correlation)

# 提取与价格相关的特征相关性
price_correlation = correlation['High Price'].sort_values(ascending=False)
print("与High Price相关性强的特征：")
print(price_correlation[abs(price_correlation) > 0.3])

# 选择与价格相关性强的特征
# selected_features = ['Package_36 inch bins', 'Package_24 inch bins', 'City Name_BOSTON', 'Package_1/2 bushel cartons', 'Variety_MINIATURE']
selected_features = price_correlation[abs(price_correlation) > 0.3].index
print(selected_features)

# 准备特征和目标变量
y = df_encoded['High Price']
X = df_encoded[selected_features].drop(columns = 'High Price')

print(X.shape, y.shape)

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 构建GBDT模型
gbdt = GradientBoostingRegressor(n_estimators=100, max_depth=4, random_state=42)

gbdt.fit(X_train, y_train)

y_pred = gbdt.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))
print("R2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

# 数据处理（先划分再处理）

In [34]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('US-pumpkins.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City Name        1757 non-null   object 
 1   Type             45 non-null     object 
 2   Package          1757 non-null   object 
 3   Variety          1752 non-null   object 
 4   Sub Variety      296 non-null    object 
 5   Grade            0 non-null      float64
 6   Date             1757 non-null   object 
 7   Low Price        1757 non-null   float64
 8   High Price       1757 non-null   float64
 9   Mostly Low       1654 non-null   float64
 10  Mostly High      1654 non-null   float64
 11  Origin           1754 non-null   object 
 12  Origin District  131 non-null    object 
 13  Item Size        1478 non-null   object 
 14  Color            1141 non-null   object 
 15  Environment      0 non-null      float64
 16  Unit of Sale     162 non-null    object 
 17  Quality       

,City Name,Type,Package,Variety,Sub Variety,Grade,Date,Low Price,High Price,Mostly Low,...,Unit of Sale,Quality,Condition,Appearance,Storage,Crop,Repack,Trans Mode,Unnamed: 24,Unnamed: 25
0,BALTIMORE,NaN,24 inch bins,NaN,NaN,NaN,4/29/17,270.0,280.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,E,NaN,NaN,NaN
1,BALTIMORE,NaN,24 inch bins,NaN,NaN,NaN,5/6/17,270.0,280.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,E,NaN,NaN,NaN
2,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,9/24/16,160.0,160.0,160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
3,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,9/24/16,160.0,160.0,160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
4,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,11/5/16,90.0,100.0,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN


In [21]:
from sklearn.model_selection import train_test_split

# 预测 'Mostly High' 价格
X = df.drop(columns=['Mostly High'])
y = df['Mostly High']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"训练集大小: {X_train.shape}")
print(f"测试集大小: {X_test.shape}")

训练集大小: (1405, 25)
测试集大小: (352, 25)


In [29]:
# 将价格列转换为数值类型（去除美元符号并转换为浮点数）
price_columns = ['Low Price', 'High Price', 'Mostly Low', 'Mostly High']
for col in price_columns:
    df[col] = df[col].replace({'\$': '', ',': ''}, regex=True).astype(float)

# 将日期列转换为日期类型
df['Date'] = pd.to_datetime(df['Date'])

In [35]:
# 查看转换后的数据类型
print(df[price_columns + ['Date']].dtypes)

# 查看前几行数据
print(df[price_columns + ['Date']].head())

Low Price      float64
High Price     float64
Mostly Low     float64
Mostly High    float64
Date            object
dtype: object
   Low Price  High Price  Mostly Low  Mostly High     Date
0      270.0       280.0       270.0        280.0  4/29/17
1      270.0       280.0       270.0        280.0   5/6/17
2      160.0       160.0       160.0        160.0  9/24/16
3      160.0       160.0       160.0        160.0  9/24/16
4       90.0       100.0        90.0        100.0  11/5/16


In [24]:
# 计算每列的缺失值比例
missing_ratio = X_train.isnull().mean()

print(missing_ratio)

City Name          0.000000
Type               0.972242
Package            0.000000
Variety            0.003559
Sub Variety        0.830605
Grade              1.000000
Date               0.000000
Low Price          0.000000
High Price         0.000000
Mostly Low         0.056228
Origin             0.000712
Origin District    0.923843
Item Size          0.168683
Color              0.359431
Environment        1.000000
Unit of Sale       0.908897
Quality            1.000000
Condition          1.000000
Appearance         1.000000
Storage            1.000000
Crop               1.000000
Repack             0.000000
Trans Mode         1.000000
Unnamed: 24        1.000000
Unnamed: 25        0.943772
dtype: float64


In [25]:
# 设置阈值，比如删除缺失值超过 50% 的列
threshold = 0.50
cols_to_drop = missing_ratio[missing_ratio > threshold].index

# 删除缺失值过多的列
X_train = X_train.drop(columns=cols_to_drop)
X_test = X_test.drop(columns=cols_to_drop)

print(f"删除了 {len(cols_to_drop)} 列缺失值过多的列：{list(cols_to_drop)}")

删除了 14 列缺失值过多的列：['Type', 'Sub Variety', 'Grade', 'Origin District', 'Environment', 'Unit of Sale', 'Quality', 'Condition', 'Appearance', 'Storage', 'Crop', 'Trans Mode', 'Unnamed: 24', 'Unnamed: 25']


In [32]:
# 再次确认数值列
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
print(f"数值列数量：{len(numerical_cols)}")
print(f"数值列：{list(numerical_cols)}")

数值列数量：3
数值列：['Low Price', 'High Price', 'Mostly Low']


In [33]:
from sklearn.impute import SimpleImputer

# 填充剩余数值列的缺失值
imputer_num = SimpleImputer(strategy='median')

X_train[numerical_cols] = pd.DataFrame(
    imputer_num.fit_transform(X_train[numerical_cols]),
    columns=numerical_cols,
    index=X_train.index
)

X_test[numerical_cols] = pd.DataFrame(
    imputer_num.transform(X_test[numerical_cols]),
    columns=numerical_cols,
    index=X_test.index
)

In [37]:
# 找出所有的分类变量列（object 类型）
categorical_cols = X_train.select_dtypes(include=['object']).columns

print("分类变量列：", list(categorical_cols))

分类变量列： ['City Name', 'Package', 'Variety', 'Date', 'Origin', 'Item Size', 'Color', 'Repack']


In [41]:
from sklearn.impute import SimpleImputer

# 创建分类变量的填充器（使用众数填充）
imputer_cat = SimpleImputer(strategy='most_frequent')

# 对分类变量进行填充
X_train[categorical_cols] = imputer_cat.fit_transform(X_train[categorical_cols])

print("分类变量的缺失值已填充完成。")

分类变量的缺失值已填充完成。


In [42]:
# 检查是否还有缺失值
print(X_train[categorical_cols].isnull().sum())
print(X_train.head())

City Name    0
Package      0
Variety      0
Date         0
Origin       0
Item Size    0
Color        0
Repack       0
dtype: int64
         City Name               Package        Variety      Date  Low Price  \
199        ATLANTA          24 inch bins       PIE TYPE  11/12/16      123.5   
700        CHICAGO  1 1/9 bushel cartons       PIE TYPE   8/26/17       20.0   
526         BOSTON          36 inch bins  BIG MACK TYPE   9/23/17      175.0   
1223      NEW YORK    1/2 bushel cartons      MINIATURE   10/1/16       20.0   
1602  PHILADELPHIA          36 inch bins    HOWDEN TYPE   10/1/16      140.0   

      High Price  Mostly Low          Origin Item Size   Color Repack  
199        135.0       123.5  NORTH CAROLINA       sml  ORANGE      N  
700         20.0        20.0        ILLINOIS       med  ORANGE      N  
526        200.0       185.0    PENNSYLVANIA      xlge  ORANGE      N  
1223        20.0        20.0            OHIO       sml   WHITE      N  
1602       140.0       140